In [1]:
# LSTM for sequence classification in the IMDB dataset
import numpy
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
df = pd.read_csv("predictive_model_dataset.csv",index_col=0)
le = preprocessing.LabelEncoder()
for column_name in df.columns:
    if df[column_name].dtype == object:
        df[column_name] = le.fit_transform(df[column_name])
    else:
        pass
y = df.prediction
features = ['Day','h_num','cSteps','h_tar','h_t_ac','daily_tar','Contextual_group','Activity Pattern']
X = df[features]
y = y.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [2]:
X_train = tf.convert_to_tensor(X_train.to_numpy().reshape(-1, len(X_train), 8))
print(X_train.shape[2])
print(y_train.shape)
X_test  = tf.convert_to_tensor(X_test.to_numpy().reshape(-1, len(X_test), 8))
y_train = tf.convert_to_tensor(y_train.to_numpy().reshape(-1, len(y_train), 1))
y_test = tf.convert_to_tensor(y_test.to_numpy().reshape(-1, len(y_test), 1))
print(X_test.shape, y_test.shape)

8
(402822,)
(1, 134274, 8) (1, 134274, 1)


In [3]:
from re import VERBOSE
from tensorflow.keras.optimizers import SGD, Adam

model = Sequential()
# model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100,return_sequences='false', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1,activation="sigmoid"))
#opt = SGD(lr=0.1, momentum=0.9)
beta_1 = 0.96
beta_2 = 0.9996
epsilon = 10 ** (-9)
opt = Adam(learning_rate=0.001, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, clipnorm=1.)
model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train,epochs=50, batch_size=5000)

# Final evaluation of the model
#trainPredict = model.predict(X_test)
#testPredict = model.predict(X_test)
#print(X_test,y_test)
#scores = model.evaluate(y_test,testPredict, verbose=1)
#print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 402822, 100)       43600     
                                                                 
 dense (Dense)               (None, 402822, 1)         101       
                                                                 
Total params: 43,701
Trainable params: 43,701
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
1/1 [==============================] - 55s 55s/step - loss: 0.2960 - accuracy: 0.5001
Epoch 2/50
1/1 [==============================] - 43s 43s/step - loss: 0.2797 - accuracy: 0.5011
Epoch 3/50
1/1 [==============================] - 42s 42s/step - loss: 0.2721 - accuracy: 0.5051
Epoch 4/50
1/1 [==============================] - 53s 53s/step - loss: 0.2656 - accuracy: 0.5071
Epoch 5/50
1/1 [==============================] - 53s 5